In [9]:
import pandas as pd
df = pd.read_csv('0522_price_volatility_ver4.csv')
df

,stock_symbol,title,date,paragraphs,1_day_change_rate
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,p.m. ET\n\nOperator\n\nGreetings and welcome ...,0.012048
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,perator\n\nWelcome to the Applied Materials ea...,-0.009111
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,"ator\n\nHello, and welcome to the DXC Technolo...",-0.169014
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,"Operator\n\nThank you for standing by, and wel...",0.021192
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,", and thank you for standing by for JD.com's f...",0.029180
...,...,...,...,...,...
985,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,"r\n\nLadies and gentlemen, thank you for stand...",0.044258
986,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,"van\n\nGood evening, everyone. If you haven't ...",0.213726
987,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,"d afternoon, ladies and gentlemen. Thank you f...",0.142463
988,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,Good day and thank you for standing by. Welcom...,-0.101322


In [12]:
binary_df = pd.read_csv('0522_price_volatility_binary_label.csv')
binary_df

,stock_symbol,title,date,paragraphs,1_day_change_rate,label
0,TTWO,Take-Two Interactive Software (TTWO) Q4 2024 E...,2024-05-16,p.m. ET\n\nOperator\n\nGreetings and welcome ...,0.012048,1
1,AMAT,Applied Materials (AMAT) Q2 2024 Earnings Call...,2024-05-16,perator\n\nWelcome to the Applied Materials ea...,-0.009111,-1
2,DXC,DXC Technology (DXC) Q4 2024 Earnings Call Tra...,2024-05-16,"ator\n\nHello, and welcome to the DXC Technolo...",-0.169014,-1
3,LSPD,Lightspeed Commerce (LSPD) Q4 2024 Earnings Ca...,2024-05-16,"Operator\n\nThank you for standing by, and wel...",0.021192,1
4,JD,JD.com (JD) Q1 2024 Earnings Call Transcript,2024-05-16,", and thank you for standing by for JD.com's f...",0.029180,1
...,...,...,...,...,...,...
980,PYCR,Paycor HCM (PYCR) Q1 2024 Earnings Call Transc...,2023-11-08,"r\n\nLadies and gentlemen, thank you for stand...",0.044258,1
981,DUOL,Duolingo (DUOL) Q3 2023 Earnings Call Transcript,2023-11-08,"van\n\nGood evening, everyone. If you haven't ...",0.213726,1
982,AFRM,Affirm (AFRM) Q1 2024 Earnings Call Transcript,2023-11-08,"d afternoon, ladies and gentlemen. Thank you f...",0.142463,1
983,CART,Instacart (CART) Q3 2023 Earnings Call Transcript,2023-11-08,Good day and thank you for standing by. Welcom...,-0.101322,-1


In [10]:
df['paragraphs'][0]

" p.m. ET\n\nOperator\n\nGreetings and welcome to the Take-Two fourth quarter and fiscal year 2024 earnings call. At this time, all participants are in a listen-only mode. A brief question-and-answer session will follow the formal presentation. [Operator instructions] As a reminder, this conference is being recorded.\n\nIt is now my pleasure to introduce your host, Nicole Shevins, senior vice president of investor relations and corporate communications. Thank you. You may begin.\n\nNicole Shevins--Senior Vice President of Investor Relations and Corporate Communications\n\nGood afternoon. Thank you for joining our conference call to discuss our results for the fourth quarter and fiscal year 2024 ended March 31, 2024. Today's call will be led by Strauss Zelnick, Take-Two's chairman and chief executive officer; Karl Slatoff, our president; and Lainie Goldstein, our chief financial officer. We will be available to answer your questions during the Q&A session following our prepared remarks.

In [11]:
%%time
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 讀取數據
train_df = df.copy()

# 文本預處理函數
def preprocess_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    return text

train_df['paragraphs'] = train_df['paragraphs'].apply(preprocess_text)

# 特徵提取 - 使用TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(train_df['paragraphs'])

# 標籤
y = train_df['1_day_change_rate']

# 拆分訓練和測試數據
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練模型
model = LinearRegression()
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 評估模型
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 查看部分預測結果
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results.head())

Mean Squared Error: 0.11763233147706523
       Actual  Predicted
213 -0.024920   0.068373
331  0.006703  -0.231148
501 -0.019166  -0.455563
309  0.015060   0.001834
88  -0.068714  -0.118451
CPU times: total: 2.25 s
Wall time: 4.73 s


In [13]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# 讀取數據
bert_df = binary_df.copy()

# BERT 模型與 tokenizer 初始化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定義特徵提取函數
def extract_features(paragraphs):
    inputs = tokenizer(paragraphs, return_tensors='pt', truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    # 使用 [CLS] 標記的向量作為句子的表示
    features = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return features

# 提取特徵並顯示進度條
features = []
for paragraph in tqdm(bert_df['paragraphs'], desc="Extracting features"):
    features.append(extract_features(paragraph))

bert_df['features'] = features

# 構建訓練和測試集
X = np.vstack(bert_df['features'].values)
y =bert_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 SVM 模型
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# 評估模型
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

Extracting features: 100%|██████████| 985/985 [02:18<00:00,  7.13it/s]


              precision    recall  f1-score   support

          -1       0.41      0.55      0.47        84
           1       0.56      0.42      0.48       113

    accuracy                           0.48       197
   macro avg       0.49      0.49      0.48       197
weighted avg       0.50      0.48      0.48       197



In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# 讀取數據
segment_df = binary_df.copy()

# BERT 模型與 tokenizer 初始化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定義特徵提取函數（分段處理）
def extract_features(paragraphs, max_length=512, stride=256):
    inputs = tokenizer(paragraphs, return_tensors='pt', truncation=True, padding=True, max_length=max_length).to(device)
    input_ids = inputs['input_ids'][0].tolist()
    
    features = []
    for i in range(0, len(input_ids), stride):
        chunk_ids = input_ids[i:i+max_length]
        if len(chunk_ids) < max_length:
            chunk_ids += [tokenizer.pad_token_id] * (max_length - len(chunk_ids))
        chunk_inputs = torch.tensor([chunk_ids]).to(device)
        with torch.no_grad():
            outputs = model(input_ids=chunk_inputs)
        chunk_features = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        features.append(chunk_features)
    
    return np.mean(features, axis=0)  # 將所有段落的特徵向量取平均

# 提取特徵並顯示進度條
features = []
for paragraph in tqdm(segment_df['paragraphs'], desc="Extracting features"):
    features.append(extract_features(paragraph))

segment_df['features'] = features

# 構建訓練和測試集
X = np.vstack(segment_df['features'].values)
y = segment_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 SVM 模型
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# 評估模型
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

Extracting features: 100%|██████████| 985/985 [02:46<00:00,  5.93it/s]


              precision    recall  f1-score   support

          -1       0.44      0.60      0.51        84
           1       0.59      0.43      0.50       113

    accuracy                           0.50       197
   macro avg       0.51      0.51      0.50       197
weighted avg       0.53      0.50      0.50       197



In [15]:
pip install sentence-transformers

   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   -- ------------------------------------- 10.2/171.5 kB ? eta -:--:--
   ------ -------------------------------- 30.7/171.5 kB 640.0 kB/s eta 0:00:01
   ------------- ------------------------- 61.4/171.5 kB 544.7 kB/s eta 0:00:01
   ------------------------ ------------- 112.6/171.5 kB 726.2 kB/s eta 0:00:01
   ------------------------------------ - 163.8/171.5 kB 893.0 kB/s eta 0:00:01
   -------------------------------------- 171.5/171.5 kB 790.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
from transformers import BertTokenizer

# 初始化tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 要轉換的句子
sentence = "This is an example sentence."

# 將句子轉換為token
tokens = tokenizer.tokenize(sentence)
print("Tokens:", tokens)

# 將token轉換為ID
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", input_ids)

# 將句子轉換為BERT模型輸入格式
inputs = tokenizer(sentence, return_tensors='pt')
print("Model Inputs:", inputs)

Tokens: ['this', 'is', 'an', 'example', 'sentence', '.']
Token IDs: [2023, 2003, 2019, 2742, 6251, 1012]
Model Inputs: {'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
